In [13]:
from datetime import datetime
import json
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AwqConfig, GPTQConfig, BitsAndBytesConfig, pipeline
from trl import SFTTrainer, SFTConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import Dataset

In [14]:
def load_model_and_tokenizer(model_name, quantization_config, device_map):
    # Load base model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map=device_map)
    
    if not tokenizer.chat_template:
        tokenizer.chat_template = """{% for message in messages %}
                {% if message['role'] == 'system' %}System: {{ message['content'] }}\n
                {% elif message['role'] == 'user' %}User: {{ message['content'] }}\n
                {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
                {% endif %}
                {% endfor %}"""
    
    # Tokenizer config
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
        
    return model, tokenizer

def preprocess_function(df, context: bool):
    if context:
        processed_data = df.apply(lambda row: {
            "prompt": [{"role": "user", "content": row["question"] + row["context"]}],
            "completion": [{"role": "assistant", "content": row["answer"]}]
        }, axis=1)
    else:
        processed_data = df.apply(lambda row: {
            "prompt": [{"role": "user", "content": row["question"]}],
            "completion": [{"role": "assistant", "content": row["answer"]}]
        }, axis=1)

    # Convert the resulta to HuggingFace Dataset
    return Dataset.from_list(processed_data.tolist())

In [15]:
DATASET_CHOICE = "arc"       # options: "arc", "boolq", "squad"
FINETUNING = "SFT"

MODEL_NAME = "Qwen/Qwen3-1.7B"

QUANT_METHOD = "QLORA"  # options: "None", "QLORA", "AWQ", "GPTQ"

device_map = {"": 0} if torch.cuda.is_available() else {"": "cpu"}

In [16]:
# ============================================================
# Dataset selection
# ============================================================

if DATASET_CHOICE == "arc":
    df = pd.read_parquet("../Datasets/test-ai2_arc.parquet").head(3)
elif DATASET_CHOICE == "boolq":
    df = pd.read_parquet("../Datasets/test-boolq.parquet")
elif DATASET_CHOICE == "squad":
    df = pd.read_parquet("../Datasets/test-squad_v2.parquet")
else:
    raise ValueError("Invalid DATASET_CHOICE")

context = DATASET_CHOICE != "arc"

print(f"Loaded dataset: {DATASET_CHOICE}")

Loaded dataset: arc


In [17]:
# --------------------------------------------
# QUANTIZATION METHOD
# --------------------------------------------

match QUANT_METHOD:
    case "QLORA":
        load_in_4bit = True # Load the model in 4-bit
        bnb_4bit_quant_type = "nf4"
        bnb_4bit_use_double_quant = True # Saves more memory at no additional performance
        bnb_4bit_compute_dtype = torch.bfloat16 


        quantization_config = BitsAndBytesConfig(
            load_in_4bit=load_in_4bit,
            bnb_4bit_quant_type=bnb_4bit_quant_type,
            bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
            bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
        )

    case "AWQ":
        bits = 4
        fuse_max_seq_len = 512
        do_fuse = True

        quantization_config = AwqConfig(
            bits=bits,
            fuse_max_seq_len=fuse_max_seq_len,
            do_fuse=do_fuse,
        )

    case "GPTQ":
        bits = 4
        v2 = True # Activate GPTQ v2
        
        quantization_config = GPTQConfig(
            bits=bits, 
            dataset=dataset, 
            tokenizer=tokenizer,
            v2=v2
        )


    case "adaround":
        raise NotImplementedError("Implement AdaRound here")

    case "brecq":
        raise NotImplementedError("Implement BRECQ here")

    case "quarot":
        raise NotImplementedError("Implement QuaRot here")

    case _:
        quantization_config = None

In [18]:
model, tokenizer = load_model_and_tokenizer(MODEL_NAME, quantization_config, device_map)
model = prepare_model_for_kbit_training(model)

lora_r = 32
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
lora_alpha = 16
lora_dropout = 0.1

config = LoraConfig(
    r=lora_r, 
    lora_alpha=lora_alpha, 
    target_modules=target_modules, 
    lora_dropout=lora_dropout, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

c:\Users\Ghiki\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ghiki\.cache\huggingface\hub\models--Qwen--Qwen3-1.7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


In [19]:
# Output folder where both checkpoints and the final fine-tuned model will be saved
model_name = MODEL_NAME.split("/")[-1]
new_model_name = f"{model_name}-{DATASET_CHOICE}_{FINETUNING}_{QUANT_METHOD}" 
output_dir = f"../Models/{new_model_name}" 

# Batch size per GPU during training (effective batch size grows with gradient accumulation)
per_device_train_batch_size = 1  

# Batch size per GPU during evaluation
per_device_eval_batch_size = 8  

# Number of steps to accumulate gradients before performing a backward/update pass.
# Useful when GPU memory is limited and batch size is very small.
gradient_accumulation_steps = 1  

# Learning rate for optimizer. Common values for fine-tuning range between 1e-5 and 1e-4.
learning_rate = 8e-5  

# Weight decay for regularization (helps prevent overfitting)
weight_decay = 0.0  

# Maximum allowed gradient norm (gradient clipping). Prevents exploding gradients.
max_grad_norm = 1.0  

# Number of full passes (epochs) over the training dataset
num_train_epochs = 1  

# Maximum training steps. If set, this overrides `num_train_epochs`.
max_steps = 1  

# Warmup ratio for the learning rate schedule (fraction of training used to "warm up")
warmup_ratio = 0.0  

# Frequency (in steps) at which training logs are reported
logging_steps = 10  

# If True, groups training samples by similar length for efficiency
group_by_length = False  

# Enables gradient checkpointing to reduce memory usage.
# Slows training slightly but allows fine-tuning larger models.
gradient_checkpointing = True  

# Concatenates multiple training examples into a single sequence to improve efficiency
packing = True  

# If True, loss is only computed on assistant responses (ignores user/system messages)
assistant_only_loss = False  

# Maximum sequence length (in tokens) for tokenized inputs
max_length = 1024  

# Preprocess the dataset into tokenized format
train_dataset = preprocess_function(df, context)


# ===============================
# Training configuration
# ===============================
sft_config = SFTConfig(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    logging_steps=logging_steps,
    packing=packing,
    assistant_only_loss=assistant_only_loss,
    max_length=max_length,
    num_train_epochs=num_train_epochs,
    max_steps=max_steps
)


# ===============================
# Trainer initialization
# ===============================
sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset, 
    processing_class=tokenizer,
)


c:\Users\Ghiki\AppData\Local\Programs\Python\Python313\Lib\site-packages\trl\trainer\sft_trainer.py:453: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
c:\Users\Ghiki\AppData\Local\Programs\Python\Python313\Lib\site-packages\trl\trainer\sft_trainer.py:495: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2' or 'kernels-community/vllm-flash-attn3'. Packing flattens batches into a single sequence, and Flash Attention is the only known attention mechanisms that reliably suppor

In [ ]:
# TODO fix percentage of trainable parameters

# Calculate total and trainable parameters
trainable_params = 0
total_params = 0
for _, param in model.named_parameters():
    total_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
percentage_trainable = trainable_params / total_params * 100

# Print a summary of the model
print("Model Summary:")
print(f"Model name: {MODEL_NAME}")
print(f"Number of parameters: {total_params / 1e9:.4f} billion")
print(f"Number of trainable parameters: {trainable_params / 1e9:.7f} billion")
print(f"Percentage of trainable parameters: {percentage_trainable:.7f}%")
print(f"Number of layers: {model.config.num_hidden_layers}")
print(f"Hidden size: {model.config.hidden_size}")

print('=========================================================================================================')

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    device = torch.device("cuda")
    current_gpu_name = torch.cuda.get_device_name(device.index)
    print(f"Using GPU: {current_gpu_name}")

    memory_stats = torch.cuda.memory_stats(device)
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = memory_stats['allocated_bytes.all.current']
    reserved_memory = memory_stats['reserved_bytes.all.current']
    free_memory = total_memory - reserved_memory

    print(f"Total VRAM: {total_memory / (1024 ** 3):.2f} GB")
else:
    device = torch.device("cpu")
    print("Using CPU")

print('=========================================================================================================')

Model Summary:
Model name: Qwen/Qwen3-1.7B
Number of parameters: 1.0288 billion
Number of trainable parameters: 0.0000000 billion
Percentage of trainable parameters: 0.0000000%
Number of layers: 28
Hidden size: 2048
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 4090
Using GPU: NVIDIA GeForce RTX 4090
Total VRAM: 23.99 GB


In [21]:
# ===============================
# Entrenamiento
# ===============================
sft_trainer.train()

Step,Training Loss


TrainOutput(global_step=1, training_loss=10.04808521270752, metrics={'train_runtime': 0.9763, 'train_samples_per_second': 1.024, 'train_steps_per_second': 1.024, 'total_flos': 2124849125376.0, 'train_loss': 10.04808521270752})

In [22]:
model = sft_trainer.model.merge_and_unload()
model.save_pretrained(output_dir, safe_serialization=True)
tokenizer.save_pretrained(output_dir)

def safe_serialize(obj):
    """Converts no serializable objects to serializable formats."""
    if isinstance(obj, (str, int, float, bool, type(None))):
        return obj
    elif isinstance(obj, (list, tuple)):
        return [safe_serialize(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: safe_serialize(value) for key, value in obj.items()}
    elif isinstance(obj, SFTConfig):
        return {k: safe_serialize(v) for k, v in obj.to_dict().items()}
    elif isinstance(obj, BitsAndBytesConfig):
        return {k: safe_serialize(v) for k, v in vars(obj).items() if not k.startswith('_')}
    elif hasattr(obj, '__dict__'):
        return {k: safe_serialize(v) for k, v in vars(obj).items() if not k.startswith('_')}
    else:
        return str(obj)
    
def drop_nulls(obj):
    """Eliminate keys with None values recursively."""
    if isinstance(obj, dict):
        return {k: drop_nulls(v) for k, v in obj.items() if v is not None}
    elif isinstance(obj, list):
        return [drop_nulls(v) for v in obj if v is not None]
    else:
        return obj


# Create complete metadata report
training_metadata = {
    "model_info": {
        "model_name": new_model_name,
        "base_model": MODEL_NAME,
        "fine_tuning_date": datetime.now().isoformat(),
        "model_type": "CausalLM",
        "has_quantization": quantization_config is not None,
        "total_params": total_params,
        "trainable_params": trainable_params,
        "percentage_trainable": percentage_trainable
    },
    "training_parameters": safe_serialize(sft_trainer.args),
    "lora_config": safe_serialize(config),
    "quantization_config": safe_serialize(quantization_config) if quantization_config else None,
    "training_stats": {
        "total_steps": sft_trainer.state.max_steps,
        "epochs_completed": sft_trainer.state.epoch,
    },
    "hardware_info": {
        "device": str(model.device),
        "dtype": str(model.dtype),
    }
}

clean_metadata = drop_nulls(training_metadata)

# Save metadata to JSON file
with open(f"{output_dir}/training_metadata.json", "w") as f:
    json.dump(clean_metadata, f, indent=4, ensure_ascii=False)

# Save model config
model.config.save_pretrained(output_dir)

print(f"Model saved in: {output_dir}")
print("File structure:")
print(f"  - model.safetensors")
print(f"  - config.json")
print(f"  - training_metadata.json")
print(f"  - tokenizer files")

c:\Users\Ghiki\AppData\Local\Programs\Python\Python313\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Model saved in: ../Models/Qwen3-1.7B-arc_SFT_QLORA
File structure:
  - model.safetensors
  - config.json
  - training_metadata.json
  - tokenizer files


In [23]:
# --------------------------------------------
# INFERENCE
# --------------------------------------------

prompt = "Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation? A Put the objects in groups. B Change the height of the ramp. C Choose different objects to roll. D Record the details of the investigation."

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map=device_map,
    use_cache=False,
    do_sample=True,
    temperature=0.7,
    max_length=200,
    pad_token_id=tokenizer.eos_token_id
)

result = pipe(prompt)
print(result[0]['generated_text'])

Device set to use cuda:0
c:\Users\Ghiki\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: expected scalar type Float but found BFloat16